In [18]:
from langchain.document_loaders import NotionDirectoryLoader
import pickle
loader = NotionDirectoryLoader("./notion_db")
docs = loader.load()

# Now, let's export this list to a binary file using pickle

In [19]:
def count_token(text: str) -> int:
    import tiktoken
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    token_count = len(encoding.encode(text))

    print(f"The text contains {token_count} tokens.")
    return token_count

In [20]:
print(len(docs))

2


In [21]:
docs[0].metadata

{'source': 'notion_db/위키 3105b855c1ef4234afbb2203e7fa069e.md'}

In [22]:
len(docs[0].page_content)
print(count_token(docs[0].page_content))

The text contains 28485 tokens.
28485


In [23]:
docs

[Document(page_content="# 위키\n\n<aside>\n💡 일타역무 교재 (민원응대만 발췌), 역무실무 교재, SQI 조사 질의답변, 고객서비스 로드맵(2022)\n\n</aside>\n\n# STEPⅠ 역무일반\n\n## CHAPTER1 여객운송관리\n\n### 1강 부산도시철도 요금표\n\n1. 부산도시철도 요금표 (2023. 10. 6. 시행)(단위 : 원)\n\n| 구 분 |  | 1구간 | 2구간 | 비고 |\n| --- | --- | --- | --- | --- |\n| 어른 | 교통카드\n모바일승차권 | 1,450 | 1,650 |  |\n|  | QR승차권 | 1,550 | 1,750 | 어른카드 100원 할증 |\n| 청소년 | 교통카드\n모바일승차권 | 1,050 | 1,200 |  |\n|  | QR승차권 | 1,150 | 1,300 | 청소년카드 100원 할증 |\n| 어린이 | 교통카드\n모바일승차권 | 무료 | 무료 |  |\n|  | QR승차권 | 700 | 800 |  |\n| 다자녀 | 교통카드 | 750 | 850 |  |\n|  | QR승차권 | 800 | 900 |  |\n- 승차권 분실 등 출장 : 어른 1,650원(최대구간 카드), 청소년 1,200원(최대구간 카드), 어린이 800원(최대구간 QR)\n- 개표 후 **3시간** 초과 : 어른 1,650원(최대구간 카드), 청소년 1,200원(최대구간 카드), 어린이 800원(최대구간 QR)\n\n### 2강 부산대중교통 요금표\n\n1. 부산대중교통 요금표 (2023. 10. 6. 시행) (단위 : 원)\n\n| 구분 | 교통카드 |  |  | 종이승차권\n (시내버스는 현금) |  |  |\n| --- | --- | --- | --- | --- | --- | --- |\n|  | 일반(만19세 이상) | 청소년(만13세~만19세 미만) | 어린이( 만6세~만13세 미만) | 일반(만19세 이상) | 청소년(만13세~만19세 미만) | 어린이( 만6세~

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 1000
chunk_overlap = 200
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap)
splits = text_splitter.split_documents(docs)
with open('../documents_export.pkl', 'wb') as f:
    pickle.dump(splits, f)

In [8]:
from dotenv import load_dotenv, find_dotenv
import os
import openai
import sys
sys.path.append('../..')
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [9]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()
persist_directory = './wiki'
ids = list(range(len(splits)))

In [10]:
db = Chroma.from_documents(documents=splits, embedding=embedding,
                           persist_directory=persist_directory)

In [ ]:
vectordb = Chroma(
    embedding_function=embedding,
    persist_directory=persist_directory
)

OSError: [Errno 30] Read-only file system: '/chroma'

In [38]:
vectordb.similarity_search('승차권', k=1)

[Document(page_content='1.  **단체승차권 처리**\n- 접속방법 : 통합 홈페이지 → 업무지원 → 홈페이지 관리자모드\n- 영수증 발매 : 수납처리 후 “발매”버튼 클릭\n- 역보관용, 승객용 영수증이 한 면에 인쇄\n- 고객보관용 영수증은 고객에게 전달, 역보관용 영수증은 역에서 보관\n- **2023.4.1. 이후  단체승차권 20명 당 1인 무임 폐지, 기존 주문제승차권(할인율 15%) 폐지**', metadata={'Header 1': 'STEPⅠ 역무일반', 'Header 2': 'CHAPTER1 여객운송관리', 'Header 3': '11강 단체승차권 처리'})]

In [24]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory='./chroma.db',
)

In [22]:
questions = {
    "역사내 시설물 문의": [
        "화장실이 어디있나요?",
        "서면 가려면 어디로 타야되나요?",
        "엘리베이터는 어디 있나요?",
        "엘리베이터 언제 고쳐요?",
        "무인민원발급기",
        "주민등록 등본 발급",
        "도서 대출기가 있나요?",
        "전기 휠체어 충전은 어떻게 하나요?",
        "휴대폰을 충전하고 싶어요"
    ],
    "역사 운영 문의": [
        "몇시까지 영업하나요?",
        "내일 첫차가 언제인가요?",
        "내일 막차가 언제인가요?",
        "물좀 주세요",
        "잠시만 이 물품좀 보관해 주세요"
    ],
    "도시철도 이용 문의": [
        "정기권은 어떻게 구매하나요?",
        "잘못 탔어요, 반대편으로 넘어가고 싶어요",
        "카드가 안되요",
        "미개표인지",
        "이미 집표된 카드인지",
        "개표 후 못들어가는 카드인지",
        "승차권을 구입하고 싶은데 현금이 없어요",
        "승차권은 현금으로만 구입 가능한가요?",
        "계좌이체로 현금을 받을 수 있나요?"
    ],
    "우대권, 복지카드.": [
        "교통카드 충전하고 싶어요",
        "우대권은 어디서 발급받나요?",
        "우대권 발급받는 기계가 어디 있나요?",
        "우대권 인식이 안됩니다.",
        "복지교통카드를 잃어버렸어요."
    ],
    "LLM이 처리할 수 없는 이례적인 상황(직원 호출 필요)": [
        "교통카드 인식이 안되요.",
        "물건을 열차내에서 습득했어요",
        "물건을 역사내에서 습득했어요",
        "물건을 잃어버렸어요",
        "물을 마시고 싶어요",
        "엘리베이터가 고장났어요",
        "사람이 쓰러져 있어요",
        "거스름돈으로 바꾸고 싶어요.",
        "휴대폰으로 탔는데 배터리가 다됐어요",
        "구간초과라고 떠요",
        "표를 잃어버렸어요."
    ]
}

In [23]:
for question in questions['도시철도 이용 문의']:
    docs = vectordb.similarity_search(question, k=3)
    result = ''
    print(len(docs))
    print('\n\n')
    print('Question -> ', question)
    print('\n\n')
    for doc in docs:
        print(doc.metadata)
        print(doc.page_content[:1000])

3



Question ->  정기권은 어떻게 구매하나요?



{'Header 1': 'STEPⅠ 역무일반', 'Header 2': 'CHAPTER1 여객운송관리', 'Header 3': '8강 정기승차권'}
1. 정기승차권 기본 정보, 종류 및 요금 (1일권, 3일권, 1개월권) (**2023. 10. 6. 시행)**
- 사용 만료된 경우 1개월권 정기권카드는 운임 보충하여 계속 사용가능
- 구입 후 1년 이내에 발생한 불량카드는 교환가능 (고객부주의로 인한 경우 제외)
- 정기승차권은 ****버스 환승 불가  
| 구분 |  | 1일권 | 3일권 | 1개월권 |
| --- | --- | --- | --- | --- |
| 운임 | 어 른 | 5,000원 | 12,000원 | 45,000원 |
|  | 청소년 | 4,000원 | 9,000원 | 33,000원 |
| 사용기간 |  | 발매 당일 영업시간
(05:00∼익일 00:50) | 보충일로부터 3일
이내 | 보충일로부터 30일
이내 |
| 사용횟수 |  | 제한 없음 | 제한 없음 | 최대 45회 |
| 구입장소 |  | 앱 | ∘어 른 : 앱, ACM
∘청소년 : 앱 | ∘어 른 : 앱, ACM
∘청소년 : 앱 |
| 잔액반환 |  | ∘사 용 : 반환 불가
∘미사용 : 메트로페이  
| ∘사 용 : 반환 불가
∘미사용
- QR : 메트로페이
- 카드 : 현금반환
| ∘ QR : 메트로페이
∘ 카드 : ACM을 통한 현금반환
- 카드구입대금 : 미반환
- 사용기간경과 : 잔여횟수 불문, 미반환
- 잔여사용횟수 : 기간 미경과 시에만 반환
ㆍ 산식 : 운임-(기본운임×사용횟수)-수수료(100원)
* 앱에서 발행한 경우 수수료 차감 없음 |  
1. 정기승차권 기간 계산 방법
- 1일의 경우 0시부터 24시까지를 말하며 24시 이후는 익일로 계산
- 예시1) 10월 1일부터 1개월 : 10월 31일까지
- 예시2) 4월 15일부터 1개월: 5월 14일까지
- 정기승차권 현금영수증 발급 방법 (역에서 발